In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls /kaggle/input/dogs-vs-cats-redux-kernels-edition

In [ ]:
!unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
!unzip /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip

In [ ]:
!ls

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate
from pathlib import Path
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.getcwd()

In [ ]:
path = Path()
print(path.ls())
train_path = path/'train'
test_path = path/'test'
print(train_path, test_path)

In [ ]:
fnames = get_image_files(train_path)
fnames[:5]

In [ ]:
np.random.seed(2)
#pat = r'/([^/]+).\d+.jpg$'
pat = r"([a-z]+).\d+.jpg$"

In [ ]:
bs = 64 #Batch size
data = ImageDataBunch.from_name_re(path, fnames, pat, ds_tfms=get_transforms(),test='test', size=224, bs=bs
                                  ).normalize(imagenet_stats)
data

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

In [ ]:
!find . -name *.pth -print

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
learn.load('stage-1');
learn.lr_find(stop_div=False, num_it=200)

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6, 1e-4))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.recorder.plot_lr()

In [ ]:
learn.save('stage-2')

In [ ]:
preds, _ = learn.get_preds(DatasetType.Test)

In [ ]:
print(preds.size())
print(preds)

In [ ]:
labels = np.argmax(preds, axis=1)
labels

In [ ]:
input_path = Path('/kaggle/input/dogs-vs-cats-redux-kernels-edition')
test_df = pd.read_csv(input_path/'sample_submission.csv')
test_df.head()

In [ ]:
ids = [int(file.stem) for file in data.test_ds.x.items]
ids

In [ ]:
submission = pd.DataFrame({'id':ids,'label':labels})
submission.head()

In [ ]:
submission = submission.sort_values(by=['id'])

In [ ]:
submission.to_csv('submission.csv', index=False)